In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import plotly 
import plotly.plotly as py 
import plotly.figure_factory as ff
from plotly.graph_objs import *
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import seaborn as sns
import scipy.stats as stats
import math
from sklearn import linear_model
from sklearn.linear_model import SGDRegressor

In [2]:
monthlyAsylumSeekers = pd.read_csv("../data/refugee_monthly.csv", sep=",", engine='python', encoding ='latin1')   
monthlyAsylumSeekers = monthlyAsylumSeekers[monthlyAsylumSeekers['Value'] != '*']
monthlyAsylumSeekers['Value'] = monthlyAsylumSeekers['Value'].apply(pd.to_numeric)
monthlyAsylumSeekers.head()

,Country / territory of asylum/residence,Origin,Year,Month,Value
0,Greece,Georgia,2008,March,140
1,Greece,Georgia,2008,April,199
2,Greece,Georgia,2008,May,210
3,Greece,Georgia,2008,June,208
4,Greece,Georgia,2008,July,224


In [3]:
monthlyAsylumSeekers.sort_values(by=['Year'])
monthlyAsylumSeekers.rename(columns={'Country / territory of asylum/residence':'Country_territoryofasylum_residence'}, inplace=True)
monthlyAsylumSeekers.head()

,Country_territoryofasylum_residence,Origin,Year,Month,Value
0,Greece,Georgia,2008,March,140
1,Greece,Georgia,2008,April,199
2,Greece,Georgia,2008,May,210
3,Greece,Georgia,2008,June,208
4,Greece,Georgia,2008,July,224


In [4]:
from sklearn import preprocessing
countryTrainer=preprocessing.LabelEncoder()
originTrainer=preprocessing.LabelEncoder()
monthTrainer=preprocessing.LabelEncoder()

Transform categorical into numerical

In [5]:

monthlyAsylumSeekers['Country_territoryofasylum_residence']=countryTrainer.fit_transform(monthlyAsylumSeekers['Country_territoryofasylum_residence'])
monthlyAsylumSeekers['Month']=monthTrainer.fit_transform(monthlyAsylumSeekers['Month'])
monthlyAsylumSeekers['Origin']=originTrainer.fit_transform(monthlyAsylumSeekers['Origin'])
monthlyAsylumSeekers[['Country_territoryofasylum_residence','Origin', 'Month']].head()

,Country_territoryofasylum_residence,Origin,Month
0,15,68,7
1,15,68,0
2,15,68,8
3,15,68,6
4,15,68,5


Test to see if it works

In [6]:
print monthTrainer.inverse_transform([0,1,2,3,4,5,6,7,8,9,10,11])
print monthTrainer.transform(['March','April','May'])
print countryTrainer.transform(['Greece'])
print originTrainer.transform(['Georgia'])

[u'April' u'August' u'December' u'February' u'January' u'July' u'June'
 u'March' u'May' u'November' u'October' u'September']
[7 0 8]
[15]
[68]


SGD **Regressor** first so y needs to be numerical value, Value.

In [7]:
X=monthlyAsylumSeekers[['Country_territoryofasylum_residence','Origin','Month']]
y=monthlyAsylumSeekers['Value']
y.head()

0    140
1    199
2    210
3    208
4    224
Name: Value, dtype: int64

Test linear regression (not important, but wanted to see)

In [8]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)
X_train.shape, X_test.shape

C:\Users\Basheer\Anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning:

This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.



((372364, 3), (124122, 3))

SGDClassifier to predict where a country of asylum based on value, month, and country of origin

In [ ]:
from sklearn.svm import SVR
clf = SVR(kernel='linear')
clf.fit(X_train, y_train)
clf.score(X_test,y_test)

In [11]:
clf.predict([[countryTrainer.transform(['Greece']),originTrainer.transform(['Afghanistan'])[0],monthTrainer.transform(['March'])[0]]])

array([ -6.32252680e+11])